In [1]:
from dotenv import load_dotenv
import os
load_dotenv('./.env')
assert os.getenv('ENGINE') is not None, "Please set the ENGINE_STRING environment variable in the .env file"

In [2]:
from omop_constructs.alchemy.condition_modifiers.modifier_joins import grade_select
from omop_constructs.semantics import registry, registry_engine
from omop_alchemy.cdm.model import Condition_Occurrence, Concept, Episode_Event
from omop_semantics.runtime.default_valuesets import runtime

print(grade_select.compile(registry_engine, compile_kwargs={"literal_binds": True}))

SELECT tumor_grade_earliest.person_id, tumor_grade_earliest.measurement_event_id, tumor_grade_earliest.meas_event_field_concept_id, tumor_grade_earliest.measurement_concept_id, tumor_grade_earliest.measurement_id, tumor_grade_earliest.measurement_date, tumor_grade_earliest.concept_name, tumor_grade_earliest.rn 
FROM (SELECT tumor_grade.person_id AS person_id, tumor_grade.measurement_event_id AS measurement_event_id, tumor_grade.meas_event_field_concept_id AS meas_event_field_concept_id, tumor_grade.measurement_concept_id AS measurement_concept_id, tumor_grade.measurement_id AS measurement_id, tumor_grade.measurement_date AS measurement_date, tumor_grade.concept_name AS concept_name, row_number() OVER (PARTITION BY tumor_grade.measurement_event_id ORDER BY tumor_grade.measurement_date ASC) AS rn 
FROM (SELECT measurement.person_id AS person_id, measurement.measurement_event_id AS measurement_event_id, measurement.meas_event_field_concept_id AS meas_event_field_concept_id, measurement.me

In [3]:
from omop_constructs.alchemy.condition_modifiers import TStage, NStage, MStage, GroupStage, SizeModifier, GradeModifier, LatModifier

In [4]:
import sqlalchemy as sa
import sqlalchemy.orm as so

In [5]:
condition_concept = so.aliased(Concept, name='condition_concept')

In [6]:
TStage.__table__

<sqlalchemy.sql.selectable.Subquery at 0x128bd42d0; t_stage_filtered>

In [7]:
print(TStage.__table__.compile(registry_engine, compile_kwargs={"literal_binds": True}))


SELECT t_stage_ranked.person_id, t_stage_ranked.stage_id, t_stage_ranked.stage_date, t_stage_ranked.stage_datetime, t_stage_ranked.stage_concept_id, t_stage_ranked.measurement_event_id, t_stage_ranked.meas_event_field_concept_id, t_stage_ranked.stage_label, t_stage_ranked.stage_type, t_stage_ranked.rn 
FROM (SELECT t_stage_initial.person_id AS person_id, t_stage_initial.stage_id AS stage_id, t_stage_initial.stage_date AS stage_date, t_stage_initial.stage_datetime AS stage_datetime, t_stage_initial.stage_concept_id AS stage_concept_id, t_stage_initial.measurement_event_id AS measurement_event_id, t_stage_initial.meas_event_field_concept_id AS meas_event_field_concept_id, t_stage_initial.stage_label AS stage_label, t_stage_initial.stage_type AS stage_type, row_number() OVER (PARTITION BY t_stage_initial.measurement_event_id ORDER BY t_stage_initial.stage_type, t_stage_initial.stage_date ASC) AS rn 
FROM (SELECT measurement.person_id AS person_id, measurement.measurement_id AS stage_id, m

In [13]:
from omop_constructs.alchemy.condition_modifiers.condition_modifier_join import modified_conditions_join
from omop_constructs.alchemy.condition_modifiers.condition_modifier_mv import ModifiedCondition

In [9]:
print(modified_conditions_join.compile(registry_engine, compile_kwargs={"literal_binds": True}))

SELECT row_number() OVER () AS mv_id, condition_occurrence.person_id, condition_occurrence.condition_start_date, condition_occurrence.condition_occurrence_id, condition_occurrence.condition_source_value, condition_occurrence.condition_concept_id, condition_concept.concept_name AS condition_concept, episode_event.episode_id AS condition_episode, t_stage_filtered.stage_id AS t_stage_id, t_stage_filtered.stage_date AS t_stage_date, t_stage_filtered.stage_concept_id AS t_stage_concept_id, t_stage_filtered.stage_label AS t_stage_label, n_stage_filtered.stage_id AS n_stage_id, n_stage_filtered.stage_date AS n_stage_date, n_stage_filtered.stage_concept_id AS n_stage_concept_id, n_stage_filtered.stage_label AS n_stage_label, m_stage_filtered.stage_id AS m_stage_id, m_stage_filtered.stage_date AS m_stage_date, m_stage_filtered.stage_concept_id AS m_stage_concept_id, m_stage_filtered.stage_label AS m_stage_label, group_stage_filtered.stage_id AS group_stage_id, group_stage_filtered.stage_date AS

In [10]:
registry['metastatic_disease'].lookup('Metastasis to heart')

35226167

In [11]:
registry['metastatic_disease'].lookup('Metastatic malignant neoplasm to epicardium')

4129922

In [12]:
runtime.condition_modifiers.condition_modifier_values.metastatic_disease

36769180

In [16]:
with so.Session(registry_engine) as session:
    ModifiedCondition.refresh_mv(session)

OperationalError: (sqlite3.OperationalError) near "REFRESH": syntax error
[SQL: REFRESH MATERIALIZED VIEW modified_conditions_mv]
(Background on this error at: https://sqlalche.me/e/20/e3q8)